In [ ]:
%cd ..

In [2]:
import sys

sys.path.append('src')

In [11]:
import json
from pathlib import Path
from itertools import chain

import numpy as np
from tqdm.auto import tqdm

import data

In [4]:
%ls -lh data

total 22G
-rw-rw-r-- 1 andy andy 5,7K окт 25 22:35 grids.json
-rw-rw-r-- 1 andy andy  28M окт 17 17:54 test.jsonl
-rw-rw-r-- 1 andy andy 2,8M ноя  1 21:58 test.npz
-rw-rw-r-- 1 andy andy 5,4M окт 26 22:17 test.txt
-rw-rw-r-- 1 andy andy  17G окт 17 17:54 train.jsonl
-rw-rw-r-- 1 andy andy 1,7G ноя  1 22:08 train.npz
-rw-rw-r-- 1 andy andy  61M окт 26 21:29 train.ref
-rw-rw-r-- 1 andy andy 3,2G окт 26 22:26 train.txt
-rw-rw-r-- 1 andy andy 1,5M окт 26 21:24 valid_500.jsonl
-rw-rw-r-- 1 andy andy 150K ноя  1 21:57 valid_500.npz
-rw-rw-r-- 1 andy andy 5,2K окт 26 21:24 valid_500.ref
-rw-rw-r-- 1 andy andy 290K окт 26 22:18 valid_500.txt
-rw-rw-r-- 1 andy andy  28M окт 17 17:54 valid.jsonl
-rw-rw-r-- 1 andy andy 2,8M ноя  1 21:57 valid.npz
-rw-rw-r-- 1 andy andy 104K окт 17 17:54 valid.ref
-rw-rw-r-- 1 andy andy 5,5M окт 26 22:17 valid.txt
-rw-rw-r-- 1 andy andy  11M ноя  1 23:45 vocabulary.csv
-rw-rw-r-- 1 andy andy 9,5M окт 17 17:53 voc.txt


In [6]:
in_path = 'data/train.jsonl'
out_path = 'data/train.txt'
out_words_path = 'data/train.ref'

In [14]:
with open(in_path) as in_file, open(out_path, 'wt') as out_file:
    for line in in_file:
        d = json.loads(line)
        line_data = [
            d['curve']['grid']['grid_name'], 
            *map(
                str, 
                chain(d['curve']['x'], d['curve']['y'], d['curve']['t'])
            )
        ]
        out_file.write(' '.join(line_data) + '\n')

In [9]:
def convert_to_bin_ds(in_path, out_path):
    coordinates, times, extra_flags, lens, words = [], [], [], [], []
    with open(in_path) as in_file:
        for line in tqdm(in_file):
            d = json.loads(line)
            t = d['curve']['t']
            if any(pt > nt for pt, nt in zip(t[:-1], t[1:])):
                continue
            words.append(d['word'])
            continue
            lens.append(len(t))
            extra_flags.append(d['curve']['grid']['grid_name'] == 'extra')
            coordinates.append(np.stack((
                np.int16(d['curve']['x']),
                np.int16(d['curve']['y']),
            )).T)
            times.append(np.int16(t))
    return words
    np.savez(
        out_path,
        coordinates=np.concatenate(coordinates),
        times=np.concatenate(times),
        extra_grid=np.array(extra_flags, bool),
        lens=np.array(lens),
    )

In [12]:
valid_words = convert_to_bin_ds('data/train.jsonl', 'data/train.npz')

0it [00:00, ?it/s]

In [15]:
len(valid_words)

5999974

In [16]:
with open(out_words_path, 'wt') as f:
    f.write('\n'.join(valid_words))

In [5]:
train_ds = data.BinaryDataset.load(Path('data/train.npz'))